In [51]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor

In [52]:
df = pd.read_csv('./Data.csv')

In [53]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

models = {'Linear Regression': LinearRegression(), 'Polynomial Regression': LinearRegression(),
'SVR (rbf)': SVR(kernel = 'rbf'), 'SVR (linear)': SVR(kernel = 'linear'), 'SVR (poly)': SVR(kernel = 'poly'),
'SVR (sigmoid)': SVR(kernel = 'sigmoid'),'Decision Tree': DecisionTreeRegressor(),
'K-NN': KNeighborsRegressor, 'Random Forest': RandomForestRegressor()}

results = pd.DataFrame(columns = ['Normal', 'Scaled'])
sc = StandardScaler()

for modelName in models:
    Scaled = 0
    if modelName == 'Polynomial Regression':
        
        for a in range(1, 10):
            poly = PolynomialFeatures(a)
            models[modelName].fit(sc.fit_transform(poly.fit_transform(X_train)), y_train)
            
            if models[modelName].score(sc.transform(poly.transform(X_test)), y_test) > Scaled:
                Scaled = models[modelName].score(sc.transform(poly.transform(X_test)), y_test)
                models[modelName].fit(poly.fit_transform(X_train), y_train)
                Normal = models[modelName].score(poly.transform(X_test),y_test)
        
    elif modelName == 'K-NN':
        
        for a in range(1, 50):
            model = models[modelName](a)
            model.fit(sc.fit_transform(X_train), y_train)
            
            if model.score(sc.transform(X_test), y_test) > Scaled:
                Scaled = model.score(sc.transform(X_test), y_test)
                Normal = model.fit(X_train, y_train).score(X_test, y_test)
 
    else:
        Normal = models[modelName].fit(X_train, y_train).score(X_test, y_test)
        Scaled = models[modelName].fit(sc.fit_transform(X_train), y_train).score(sc.transform(X_test), y_test)
        
    results.loc[modelName] = dict(Normal = f'{round(Normal * 100, 2)}%', Scaled = f'{round(Scaled * 100, 2)}%')
    
results.sort_values('Scaled', ascending = False)

,Normal,Scaled
Random Forest,96.22%,96.24%
K-NN,94.66%,95.18%
Polynomial Regression,93.95%,94.48%
SVR (rbf),36.6%,94.02%
Linear Regression,92.92%,92.92%
Decision Tree,92.83%,92.85%
SVR (linear),92.78%,92.81%
SVR (poly),73.32%,77.54%
SVR (sigmoid),1.23%,-7641.97%
